In [1]:
import numpy as np
import torch
from sep_bk_nn import SepBKNN

In [2]:
loaded_model = SepBKNN.load_model('models/sep_bk_nn_test_2.pth', device=torch.device('cuda'))

/global/cfs/cdirs/des/kunhaoz/projects/seperate_bk/sep_bk_nn/sep_bk_nn.py:191: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filepath, map_location=d

Model loaded from models/sep_bk_nn_test_2.pth


In [6]:
loaded_model.evaluate(1,1,1)

TypeError: evaluate() takes 2 positional arguments but 4 were given